In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import os

import sklearn.linear_model as linear_models
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate

import xgboost as xgb
import sklearn.model_selection as ms

In [3]:
# load your dataset
data_dir = os.path.join(os.getcwd(), 'data_files')

target_df = pd.read_csv(os.path.join(data_dir, 'targets.csv'), parse_dates=['USAGE_DATE'], index_col='USAGE_DATE')
predictor_df = pd.read_csv(os.path.join(data_dir, 'predictors.csv'), parse_dates=['USAGE_DATE'], index_col='USAGE_DATE')
predictor_df['Holiday'] = predictor_df['Holiday'].astype(int)

df = target_df.join(predictor_df)
df = df.sort_index(ascending=False)
df.dropna(inplace=True)

In [4]:
region = 'SoCal' # can be SD or SoCal
target = df[region + '_TOTAL']

predictor_dict = \
{'set_1': [region + '_HDD',region + '_t-1_HDD', region + '_t-2_HDD', region + '_t-3_HDD',
                 region + '_CDD',region + '_t-1_CDD', region + '_t-2_CDD', 
                 region+'_HDD_7', region + '_t-1_HDD_7', region + '_t-2_HDD_7',
               'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec',
               'Sat','Sun','Holiday'],
'set_2': [region + '_HDD',region + '_t-1_HDD', region + '_t-2_HDD',
                 region + '_CDD',region + '_t-1_CDD', 
                 region+'_HDD_7', region + '_t-1_HDD_7', 
               'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec',
               'Sat','Sun','Holiday'],
'set_3': [region + '_HDD',region + '_t-1_HDD', region + '_t-2_HDD', region + '_t-3_HDD',
                 region + '_CDD',region + '_t-1_CDD', region + '_t-2_CDD', 
                 region+'_HDD_7', region + '_t-1_HDD_7', region + '_t-2_HDD_7',
               'day_of_year_sine', 'day_of_year_cosine',
               'Sat','Sun','Holiday'],
'set_4': [region + '_HDD',region + '_t-1_HDD', region + '_t-2_HDD',
                 region + '_CDD',region + '_t-1_CDD',
                 region+'_HDD_7', region + '_t-1_HDD_7',
               'day_of_year_sine', 'day_of_year_cosine',
               'Sat','Sun','Holiday'],
'set_5': [region + '_HDD',region + '_t-1_HDD', region + '_t-2_HDD', region + '_t-3_HDD',
                 region + '_CDD',region + '_t-1_CDD', region + '_t-2_CDD', 
                 region+'_HDD_7', region + '_t-1_HDD_7', region + '_t-2_HDD_7',
               'month_sine', 'month_cosine',
               'Sat','Sun','Holiday'],
'set_6': [region + '_HDD',region + '_t-1_HDD', region + '_t-2_HDD',
                 region + '_CDD',region + '_t-1_CDD',
                 region+'_HDD_7', region + '_t-1_HDD_7',
               'month_sine', 'month_cosine',
               'Sat','Sun','Holiday'],
}

In [ ]:
evaluation_df = pd.DataFrame()
for key, value in predictor_dict.items(): 
    
    xgb_model = xgb.XGBRegressor()
    predictors = df[value]
    # hyperparameter tuning for XGBoost 
    params = {
            'learning_rate'  : [0.10, 0.20, 0.30 ], #increasing value decreases computation time but possibly accuracy
            'min_child_weight': [3, 5, 8, 10], # larger = more conservative
            'gamma': [0.5, 1, 1.5, 2, 5], # loss reduction threshold to make a split. larger =  more conservative
            'colsample_bytree': [0.6, 0.75, 0.9], # %data to sample per tree. Smaller proportion of data = reduce overfitting
            'max_depth': [3, 4, 5, 6, 7] # % max depth of each tree. Shallower tree depth = reduce overfitting
            }

    xgb_grid = ms.GridSearchCV(xgb_model, param_grid=params, 
                             scoring='neg_mean_absolute_error', verbose=3 ) #default is 5-fold CV
    xgb_grid.fit(predictors, target)
    
    scoring = ['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'explained_variance']
    xgb_scores = cross_validate(xgb_grid.best_estimator_, predictors, target, cv=5, scoring = scoring)
    
    analyses = dict(zip(scoring, [np.mean(xgb_scores['test_' + metric]) for metric in scoring]))
    analyses_df = pd.DataFrame.from_dict(analyses, orient='index').transpose()
    analyses_df.index = [key]
    evaluation_df = pd.concat([evaluation_df, analyses_df])

Fitting 5 folds for each of 900 candidates, totalling 4500 fits
[CV] colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3, score=-20.280, total=   0.1s
[CV] colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3, score=-28.799, total=   0.1s
[CV] colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3, score=-27.704, total=   0.2s
[CV] colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3, score=-30.149, total=   0.1s
[CV] colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=3, score=-23.755, total=   0.1s
[CV] colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=5 
[CV]  colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=5, score=-21.429, total=   0.1s
[CV] colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=5 
[CV]  colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=5, score=-28.723, total=   0.1s
[CV] colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=5 
[CV]  colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=3, min_child_weight=5, score=-26.365, total=   0.1s
[CV] colsample_bytree=0.6, gamma=0.5, le

...

[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.1, max_depth=7, min_child_weight=10, score=-24.628, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.1, max_depth=7, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.1, max_depth=7, min_child_weight=10, score=-30.062, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.1, max_depth=7, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.1, max_depth=7, min_child_weight=10, score=-28.035, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.1, max_depth=7, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.1, max_depth=7, min_child_weight=10, score=-21.356, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=3, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=3, min_child_weight=3, score=-21.516, total=   0.1s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0

[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=4, min_child_weight=10, score=-29.068, total=   0.1s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=4, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=4, min_child_weight=10, score=-26.246, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=4, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=4, min_child_weight=10, score=-21.691, total=   0.1s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=5, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=5, min_child_weight=3, score=-19.842, total=   0.4s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=5, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=5, min_child_weight=3, score=-26.509, total=   0.3s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2

[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=6, min_child_weight=10, score=-30.490, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=6, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=6, min_child_weight=10, score=-26.623, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=6, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=6, min_child_weight=10, score=-22.572, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=7, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=7, min_child_weight=3, score=-21.426, total=   0.3s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=7, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.2, max_depth=7, min_child_weight=3, score=-27.163, total=   0.3s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.2

[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=3, min_child_weight=10, score=-26.729, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=3, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=3, min_child_weight=10, score=-22.231, total=   0.1s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=4, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=4, min_child_weight=3, score=-21.424, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=4, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=4, min_child_weight=3, score=-27.619, total=   0.1s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=4, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=4, min_child_weight=3, score=-34.006, total=   0.1s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, 

[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=5, min_child_weight=10, score=-26.397, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=5, min_child_weight=10 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=5, min_child_weight=10, score=-21.867, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=6, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=6, min_child_weight=3, score=-18.671, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=6, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=6, min_child_weight=3, score=-28.075, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=6, min_child_weight=3 
[CV]  colsample_bytree=0.6, gamma=2, learning_rate=0.3, max_depth=6, min_child_weight=3, score=-32.950, total=   0.2s
[CV] colsample_bytree=0.6, gamma=2, learning_rate=0.3, 

In [21]:
evaluation_df

,r2,neg_mean_absolute_error,neg_mean_squared_error,explained_variance
set_1,0.965968,-42.144540,-4362.224744,0.967429
set_2,0.964920,-43.443220,-4511.951183,0.966799
set_3,0.966489,-41.526248,-4313.099112,0.968591
set_4,0.965755,-42.269499,-4401.330832,0.968357
set_5,0.966941,-41.359014,-4275.287602,0.968851
set_6,0.965518,-43.423602,-4457.664332,0.967778
